In [17]:
import glob
import scipy
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chisquare
from natsort import natsorted
from scipy.stats import describe
from scipy.signal import filtfilt
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
import matplotlib

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', size=12) 
%config InlineBackend.figure_format = 'svg'

In [18]:
path = '/Users/patrick/Documents/phd/GitHub/tikhonov/'

In [19]:
g4iso_files = natsorted(glob.glob(path+"g4out/*KE_*_iso.csv"))

def cnum(xbinw,ebinw):
    R = []
    D = []

    for i in range(5,len(g4iso_files),ebinw): # 100 keV to 1 MeV energy range in steps of 100 keV
        R.append(np.loadtxt(open(g4iso_files[i], "r"), delimiter=",",skiprows=9)[30:2000:xbinw,3]/5000000.0)
    R = np.transpose(np.array(R))
    RR = R

    return(np.linalg.cond(R))

In [20]:
g4iso_files = natsorted(glob.glob(path+"g4out/*KE_*_iso.csv"))
R = []
D = []

for i in range(5,len(g4iso_files)-101,10): # 100 keV to 1 MeV energy range in steps of 100 keV
    R.append(np.loadtxt(open(g4iso_files[i], "r"), delimiter=",",skiprows=9)[30:400:1,3]/5000000.0)
R = np.transpose(np.array(R))
RR = R
for j in range(0,370): # X-ray spectra from 0 to 600 keV
    if (np.linalg.norm(R[j,:] > 0)):
        D.append((1.0/np.linalg.norm(R[j,:]))) # Diagonal scaling as a left pre-conditioner
    else:
        D.append(1)

DD = np.diag(D)
RT = np.matmul(DD,R)

S = []
for k in range(0,R.shape[1]):
    #S.append(1.0/(k+1)**2) # right pre-conditioner towards an exponential
    S.append(1.0/np.sum(R[:,k]))
    #S.append(alexa_model_exp[k])
    #S.append(1)
SS = np.diag(S)
RS = np.matmul(RT,SS)

print('response condition number: ' + str(np.linalg.cond(R)))
print('response condition number after preconditioning: ' + str(np.linalg.cond(RS)))

elow_keV = 50
ehigh_keV = 1000
xspace_estep_keV = 1
espace_estep_keV = 10

espace = np.linspace(elow_keV,ehigh_keV,R.shape[1])
xspace = np.linspace(30,400,R.shape[0])

response condition number: 2859.3284534841096
response condition number after preconditioning: 44.79055802548675


In [21]:
# generate synthetic data 

e0 = np.linspace(30, 299, R.shape[0])
e1 = np.linspace(100, ehigh_keV, R.shape[1])

def bg(total):
    beam = np.full(len(espace), 0)
    signal = np.full(len(xspace), 0)
    for e in range(0, e0.shape[0]): # normally distributed noise
        signal[e] = abs(signal[e] + np.random.normal(signal[e],np.sqrt(total/len(espace))))
    return((signal,beam))

def ex(fold, elow, ehigh, total):
    beam = np.exp(-e1/fold)
    beam[0:(np.abs(e1 - elow)).argmin()] = 0.0
    beam[(np.abs(e1 - ehigh)).argmin():-1] = 0.0
    integral = np.trapz(beam,e1)
    beam = beam*total/integral # beam should integrate to total e-/cm^2/s
    
    signal = np.matmul(R,beam)
    
    for e in range(0, e0.shape[0]):
        signal[e] = abs(signal[e] + np.random.normal(signal[e],np.sqrt(signal[e])))
        
    return(signal,beam)

def gauss(mu, sig, elow, ehigh, total):
    beam = total*np.exp(-np.power(e1 - mu, 2.) / (2 * np.power(sig, 2.)))
    beam[0:(np.abs(e1 - elow)).argmin()] = 0.0 # low and high energy cutoffs
    beam[(np.abs(e1 - ehigh)).argmin():-1] = 0.0
    integral = np.trapz(beam, x=e1)
    beam = beam*total/integral # beam should integrate to total e-/cm^2/s
    signal = np.matmul(R,beam)
    for e in range(0, e0.shape[0]): 
        signal[e] = np.random.normal(signal[e], scale=np.sqrt(signal[e]))
    return(signal,beam)

def mono(e, total):
    beam = np.full(e1.shape[0], 0.0)
    beam[(np.abs(e1 - e)).argmin()] = total
    signal = np.matmul(R,beam) # beam should integrate to total e-/cm^2/s
    for e in range(0, e0.shape[0]): 
        signal[e] = np.random.normal(signal[e], scale=np.sqrt(signal[e]))
    return((signal,beam))

In [22]:
beam0 = ex(100,0,1000,1e8)
np.savetxt('beam_ex_100_0_1000_1e10_4.out', beam0[0], delimiter=',') 
np.savetxt('beam_ex_100_0_1000_1e10_5.out', beam0[1], delimiter=',') 
np.savetxt('xspace_2',xspace,delimiter=',')
np.savetxt('espace_2',espace,delimiter=',')
np.savetxt('R2',R,delimiter=',')

In [84]:
beam00 = np.loadtxt('beam_ex_100_0_1000_1e10_0.out')
beam01 = np.loadtxt('beam_ex_100_0_1000_1e10_1.out')

beam10 = np.loadtxt('beam_ex_100_0_1000_1e10_2.out')
beam11 = np.loadtxt('beam_ex_100_0_1000_1e10_3.out')

beam20 = np.loadtxt('beam_ex_100_0_1000_1e10_4.out')
beam21 = np.loadtxt('beam_ex_100_0_1000_1e10_5.out')


xspace0 = np.loadtxt('xspace_0')
espace0 = np.loadtxt('espace_0')

xspace1 = np.loadtxt('xspace_1')
espace1 = np.loadtxt('espace_1')

xspace2 = np.loadtxt('xspace_2')
espace2 = np.loadtxt('espace_2')

R0 = np.loadtxt('R0',delimiter=',')
R1 = np.loadtxt('R1',delimiter=',')
R2 = np.loadtxt('R2',delimiter=',')


fig = plt.figure(figsize=(6.52437527778,9))
gs = plt.GridSpec(nrows=4, ncols=1,hspace=.7,wspace=.3)


ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax4 = fig.add_subplot(gs[2])
ax6 = fig.add_subplot(gs[3])

ax1.semilogy(xspace0,beam00)

ax2.plot(espace2,beam21, label='assumed')
ax2.plot(espace2,np.matmul(np.linalg.pinv(R2),beam20)/2.0,label='retrieved')

ax4.plot(espace1,beam11, label='assumed')
ax4.plot(espace1,np.matmul(np.linalg.pinv(R1),beam10)/2,label='retrieved')

ax6.plot(espace0,beam01,label='assumed')
ax6.plot(espace0,np.matmul(np.linalg.pinv(R0),beam00)/2,label='retrieved')

ax1.set_ylabel('counts/s/keV')
ax1.set_xlabel('X-ray Energy (KeV)')
ax6.set_xlabel('X-ray Energy (KeV)')

ax2.set_ylabel('e- / KeV',labelpad=10)
ax4.set_ylabel('e- / KeV',labelpad=10)
ax6.set_ylabel('e- / KeV',labelpad=10)

ax2.legend()
ax4.legend()
ax6.legend()

ax2.text(900,0.5e6,"(A)")
ax4.text(900,0.2e6,"(B)")
ax6.text(900,-.4e7,"(C)")

plt.savefig('direct_inversion_example7.pdf')  

